In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Flatten,Dropout

df = pd.read_json("../input/milling-dataset/ipt_12.json")
df.head()

,case,run,VB,DOC,time,material,feed,smcAC,smcDC,vib_table,vib_spindle,AE_table,AE_spindle
0,1,1,0.00,1.5,2,1,0.5,"[-0.0170898438, 0.263671875, 0.2075195312, 0.3...","[0.625, 0.810546875, 0.7812500000000001, 0.849...","[0.078125, 0.0854492188, 0.078125, 0.073242187...","[0.3149414062, 0.3015136719, 0.3039550781, 0.3...","[0.0872802734, 0.0982666016, 0.0921630859, 0.0...","[0.1037597656, 0.1232910156, 0.1049804688, 0.1..."
1,1,2,NaN,1.5,4,1,0.5,"[0.3076171875, 0.3125, 0.1953125, 0.1879882812...","[0.6689453125, 0.6787109375, 0.859375, 1.12304...","[0.0756835938, 0.08056640620000001, 0.078125, ...","[0.3015136719, 0.3088378906, 0.2990722656, 0.3...","[0.0866699219, 0.0897216797, 0.0946044922, 0.1...","[0.0994873047, 0.1037597656, 0.1080322266, 0.1..."
2,1,3,NaN,1.5,6,1,0.5,"[-0.7250976562, -0.7153320312, -0.5297851562, ...","[0.9130859375, 0.8349609375, 0.908203125000000...","[0.0830078125, 0.078125, 0.09033203120000001, ...","[0.2954101562, 0.2966308594, 0.29296875, 0.290...","[0.0927734375, 0.1007080078, 0.0933837891, 0.0...","[0.1049804688, 0.1190185547, 0.109863281200000..."
3,1,4,0.11,1.5,7,1,0.5,"[0.1123046875, 0.009765625, -0.1513671875, -0....","[0.1318359375, 0.3955078125, 0.7568359375, 0.8...","[0.0830078125, 0.0756835938, 0.0659179688, 0.0...","[0.3161621094, 0.3112792969, 0.302734375, 0.32...","[0.1129150391, 0.0994873047, 0.1049804688, 0.1...","[0.1397705078, 0.12145996090000001, 0.12451171..."
4,1,5,NaN,1.5,11,1,0.5,"[-0.1220703125, -0.1635742188, -0.3686523438, ...","[0.44921875, 0.6640625, 0.6689453125, 0.747070...","[0.107421875, 0.107421875, 0.1025390625, 0.100...","[0.2844238281, 0.2893066406, 0.2844238281, 0.2...","[0.0958251953, 0.09765625, 0.09765625, 0.09460...","[0.1104736328, 0.1135253906, 0.109863281200000..."


In [2]:
print(df.shape)
print(df.columns)

for col in df.columns:
    if df[col].dtypes == 'object':
        count = 0
        for val in df[col]:
            nan_sum = 0
            nan_sum = np.isnan(val).sum()
            count += nan_sum
        print(col+": ", df[col].dtypes, ",", count)
    else:
        print(col+": ", df[col].dtypes, ",", df[col].isna().sum(axis=0))
        
for col in df.columns:
    if df[col].dtypes != 'object':
        print(col+": ", df[col].unique())
        
df.dropna(axis=0, how="any", inplace=True)
print(df.shape)

target=df["VB"]
df = df[df.columns.drop("VB")]
print(df.shape)

df.reset_index()

(167, 13)
Index(['case', 'run', 'VB', 'DOC', 'time', 'material', 'feed', 'smcAC',
       'smcDC', 'vib_table', 'vib_spindle', 'AE_table', 'AE_spindle'],
      dtype='object')
case:  int64 , 0
run:  int64 , 0
VB:  float64 , 21
DOC:  float64 , 0
time:  int64 , 0
material:  int64 , 0
feed:  float64 , 0
smcAC:  object , 0
smcDC:  object , 0
vib_table:  object , 0
vib_spindle:  object , 0
AE_table:  object , 0
AE_spindle:  object , 0
case:  [ 1  2  3  4  9 10 11 12  5  6  7  8 13 14 15 16]
run:  [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
VB:  [0.    nan 0.11 0.2  0.24 0.29 0.28 0.38 0.4  0.43 0.45 0.5  0.44 0.08
 0.14 0.15 0.18 0.22 0.26 0.31 0.48 0.55 0.13 0.17 0.19 0.23 0.33 0.36
 0.35 0.49 0.1  0.27 0.47 0.64 0.81 0.04 0.16 0.25 0.53 0.7  0.07 0.09
 0.12 0.37 0.42 0.57 0.65 0.68 0.76 0.05 0.32 0.58 0.74 0.34 0.46 0.3
 0.62 0.56 0.83 0.92 1.07 1.3  1.53 0.6  1.14]
DOC:  [1.5  0.75]
time:  [  2   4   6   7  11  15  19  22  26  29  32  35  38  41  44  46  48   3


,index,case,run,DOC,time,material,feed,smcAC,smcDC,vib_table,vib_spindle,AE_table,AE_spindle
0,0,1,1,1.5,2,1,0.50,"[-0.0170898438, 0.263671875, 0.2075195312, 0.3...","[0.625, 0.810546875, 0.7812500000000001, 0.849...","[0.078125, 0.0854492188, 0.078125, 0.073242187...","[0.3149414062, 0.3015136719, 0.3039550781, 0.3...","[0.0872802734, 0.0982666016, 0.0921630859, 0.0...","[0.1037597656, 0.1232910156, 0.1049804688, 0.1..."
1,3,1,4,1.5,7,1,0.50,"[0.1123046875, 0.009765625, -0.1513671875, -0....","[0.1318359375, 0.3955078125, 0.7568359375, 0.8...","[0.0830078125, 0.0756835938, 0.0659179688, 0.0...","[0.3161621094, 0.3112792969, 0.302734375, 0.32...","[0.1129150391, 0.0994873047, 0.1049804688, 0.1...","[0.1397705078, 0.12145996090000001, 0.12451171..."
2,5,1,6,1.5,15,1,0.50,"[0.3295898438, 0.3076171875, 0.2709960938, 0.2...","[0.68359375, 0.87890625, 0.83984375, 0.9033203...","[0.0708007812, 0.0732421875, 0.0732421875, 0.0...","[0.3076171875, 0.3100585938, 0.3063964844, 0.3...","[0.1037597656, 0.0927734375, 0.0891113281, 0.0...","[0.1202392578, 0.10681152340000001, 0.09765625..."
3,6,1,7,1.5,19,1,0.50,"[-0.5786132812, -0.5834960938, -0.5224609375, ...","[1.3818359375, 1.396484375, 1.40625, 1.4208984...","[0.0659179688, 0.0659179688, 0.068359375, 0.06...","[0.322265625, 0.2990722656, 0.3137207031, 0.29...","[0.0909423828, 0.0726318359, 0.0823974609, 0.0...","[0.1239013672, 0.1013183594, 0.111694335900000..."
4,7,1,8,1.5,22,1,0.50,"[0.3002929688, 0.2563476562, 0.1708984375, 0.0...","[1.435546875, 1.4404296875, 1.4453125, 1.44531...","[0.0610351562, 0.0659179688, 0.0610351562, 0.0...","[0.3088378906, 0.3039550781, 0.3063964844, 0.2...","[0.0927734375, 0.0994873047, 0.09765625, 0.091...","[0.1086425781, 0.1245117188, 0.1141357422, 0.1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,159,15,6,1.5,16,2,0.25,"[-0.1953125, -0.2294921875, -0.0439453125, 0.0...","[1.4013671875, 1.40625, 1.40625, 1.4013671875,...","[0.0634765625, 0.0610351562, 0.0634765625, 0.0...","[0.2856445312, 0.2880859375, 0.2807617188, 0.2...","[0.0836181641, 0.0866699219, 0.0854492188, 0.1...","[0.0939941406, 0.1031494141, 0.1019287109, 0.1..."
142,160,15,7,1.5,19,2,0.25,"[-0.68359375, -0.693359375, -0.546875, -0.3613...","[1.376953125, 1.38671875, 1.4013671875, 1.4111...","[0.056152343800000004, 0.0634765625, 0.0585937...","[0.2612304688, 0.263671875, 0.2697753906, 0.26...","[0.1025390625, 0.1141357422, 0.107421875, 0.10...","[0.1232910156, 0.1293945312, 0.1318359375, 0.1..."
143,163,16,3,1.5,3,2,0.50,"[-0.2001953125, -0.009765625, 0.0830078125, 0....","[1.40625, 1.4013671875, 1.396484375, 1.3867187...","[0.0659179688, 0.0708007812, 0.0634765625, 0.0...","[0.2795410156, 0.2844238281, 0.3015136719, 0.2...","[0.1147460938, 0.0952148438, 0.1019287109, 0.0...","[0.1397705078, 0.1123046875, 0.1202392578, 0.1..."
144,165,16,5,1.5,6,2,0.50,"[-0.205078125, -0.2392578125, -0.15625, 0.1269...","[1.3818359375, 1.38671875, 1.38671875, 1.38183...","[0.068359375, 0.05859375, 0.0708007812, 0.0659...","[0.2893066406, 0.3002929688, 0.2917480469, 0.3...","[0.0988769531, 0.0964355469, 0.0897216797, 0.0...","[0.1147460938, 0.1123046875, 0.106811523400000..."


In [3]:
df1=df.drop(["smcDC","vib_table","vib_spindle","AE_table","AE_spindle"],axis=1)
df1=df1.explode("smcAC")    
df2=df.drop(['case', 'run',   'DOC', 'time', 'material', 'feed', 'smcAC',
        'vib_table', 'vib_spindle', 'AE_table', 'AE_spindle'],axis=1)
df2=df2.explode("smcDC")
df3=df.drop(['case', 'run',   'DOC', 'time', 'material', 'feed', 'smcAC',
       'smcDC', 'vib_spindle', 'AE_table', 'AE_spindle'],axis=1)
df3=df3.explode("vib_table")
df4=df.drop(['case', 'run',   'DOC', 'time', 'material', 'feed', 'smcAC',
       'smcDC', 'vib_table',  'AE_table', 'AE_spindle'],axis=1)
df4=df4.explode("vib_spindle")
df5=df.drop(['case', 'run',   'DOC', 'time', 'material', 'feed', 'smcAC',
       'smcDC', 'vib_table', 'vib_spindle', 'AE_spindle'],axis=1)
df5=df5.explode("AE_table")
df6=df.drop(['case', 'run',   'DOC', 'time', 'material', 'feed', 'smcAC',
       'smcDC', 'vib_table', 'vib_spindle', 'AE_table',],axis=1)
df6=df6.explode("AE_spindle")

df1['smcDC']=df2['smcDC']
df1['vib_table']=df3['vib_table']
df1['vib_spindle']=df4['vib_spindle']
df1['AE_table']=df5['AE_table']
df1['AE_spindle']=df6['AE_spindle']

In [4]:
df1.head()

,case,run,DOC,time,material,feed,smcAC,smcDC,vib_table,vib_spindle,AE_table,AE_spindle
0,1,1,1.5,2,1,0.5,-0.01709,0.625,0.078125,0.314941,0.08728,0.10376
0,1,1,1.5,2,1,0.5,0.263672,0.810547,0.085449,0.301514,0.098267,0.123291
0,1,1,1.5,2,1,0.5,0.20752,0.78125,0.078125,0.303955,0.092163,0.10498
0,1,1,1.5,2,1,0.5,0.302734,0.849609,0.073242,0.300293,0.095215,0.111084
0,1,1,1.5,2,1,0.5,0.239258,1.098633,0.083008,0.299072,0.083008,0.092163


In [5]:
l=np.zeros((146,9000,12))
for i in range(146):
    for j in range(9000):
        ser = df1.iloc[j]
        l1 = ser.to_numpy(dtype=np.float32)
        l[i][j] = l1

In [6]:
l.shape

(146, 9000, 12)

In [7]:
y = target.to_numpy()
print(y)

[0.   0.11 0.2  0.24 0.29 0.28 0.29 0.38 0.4  0.43 0.45 0.5  0.44 0.08
 0.14 0.14 0.14 0.15 0.18 0.22 0.26 0.31 0.38 0.43 0.48 0.55 0.   0.13
 0.13 0.17 0.19 0.2  0.23 0.23 0.26 0.28 0.33 0.36 0.44 0.55 0.08 0.13
 0.2  0.31 0.35 0.4  0.49 0.   0.1  0.14 0.19 0.27 0.38 0.47 0.64 0.81
 0.   0.04 0.08 0.16 0.25 0.36 0.43 0.47 0.53 0.7  0.   0.04 0.07 0.07
 0.08 0.09 0.12 0.16 0.18 0.2  0.23 0.26 0.31 0.37 0.42 0.47 0.57 0.65
 0.68 0.76 0.05 0.08 0.12 0.17 0.2  0.24 0.32 0.4  0.45 0.49 0.58 0.65
 0.   0.16 0.29 0.44 0.53 0.74 0.   0.   0.09 0.13 0.22 0.24 0.34 0.46
 0.   0.18 0.3  0.44 0.62 0.1  0.13 0.17 0.32 0.38 0.49 0.56 0.68 0.83
 0.92 1.07 1.3  1.53 0.09 0.17 0.24 0.35 0.6  0.81 1.14 0.15 0.28 0.37
 0.48 0.56 0.7  0.24 0.4  0.62]


In [8]:
X_train, X_val, y_train, y_val = train_test_split(l, y, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2)

In [9]:
model = Sequential()
model.add(LSTM(100,
            return_sequences=True,
            input_shape=(X_train.shape[1],12)))
model.add(LSTM(200, return_sequences=True))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
#model.add(LSTM(20))
model.add(Dense(1))

model.compile(optimizer='adam', loss="mean_squared_error")

2022-05-21 12:26:54.366192: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-21 12:26:54.500191: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-21 12:26:54.500934: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-21 12:26:54.502819: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [10]:
history = model.fit(X_train,y_train,epochs=100, validation_data=(X_val, y_val), validation_steps = 20 ,batch_size=9000)

y_pred = model.predict(X_test)
# y_pred = np.where(y_pred > 0.5,1,0)

2022-05-21 12:26:57.754501: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100


2022-05-21 12:27:01.068150: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


1/1 [==============================] - 7s 7s/step - loss: 0.4291 - val_loss: 590295.7500
Epoch 2/100
1/1 [==============================] - 1s 1s/step - loss: 590257.0625
Epoch 3/100
1/1 [==============================] - 1s 1s/step - loss: 524.8947
Epoch 4/100
1/1 [==============================] - 1s 1s/step - loss: 240.5196
Epoch 5/100
1/1 [==============================] - 1s 1s/step - loss: 269.5645
Epoch 6/100
1/1 [==============================] - 1s 1s/step - loss: 203.3208
Epoch 7/100
1/1 [==============================] - 1s 1s/step - loss: 106.2212
Epoch 8/100
1/1 [==============================] - 1s 1s/step - loss: 29.0589
Epoch 9/100
1/1 [==============================] - 1s 1s/step - loss: 13078.3340
Epoch 10/100
1/1 [==============================] - 1s 1s/step - loss: 0.1890
Epoch 11/100
1/1 [==============================] - 1s 1s/step - loss: 0.1890
Epoch 12/100
1/1 [==============================] - 1s 1s/step - loss: 0.1891
Epoch 13/100
1/1 [=======================

In [11]:
print(y_pred)

[[-0.00324198]
 [-0.00324198]
 [-0.00324198]
 [-0.00324198]
 [-0.00324198]
 [-0.00324198]
 [-0.00324198]
 [-0.00324198]
 [-0.00324198]
 [-0.00324198]
 [-0.00324198]
 [-0.00324198]
 [-0.00324198]
 [-0.00324198]
 [-0.00324198]
 [-0.00324198]
 [-0.00324198]
 [-0.00324198]
 [-0.00324198]
 [-0.00324198]
 [-0.00324198]
 [-0.00324198]
 [-0.00324198]
 [-0.00324198]]


In [12]:
y_pred = y_pred.flatten()
# cm = metrics.confusion_matrix(y_test, y_pred[:,0])
mse = metrics.mean_squared_error(y_test, y_pred)
# print(cm)
print(mse)

0.19425746170808122
